In [ ]:
!pip install -r /requirements.txt

### eval on pretrained model!

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset
import evaluate
import numpy as np
import nltk
from tqdm import tqdm

# Ensure NLTK punkt is downloaded (for ROUGE)
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

# Load base model and tokenizer
model_name = "google/pegasus-arxiv"  # or your preferred base model
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:  73%|#######2  | 1.65G/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# Load your evaluation data
dataset = load_dataset("ccdv/arxiv-summarization", split="validation[:100]")  # replace with your dataset

dataset

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00003-of-00015.parquet:   0%|          | 0.00/227M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'abstract'],
    num_rows: 100
})

In [ ]:
val_data = dataset[:20]
type(val_data), type(dataset)

(dict, datasets.arrow_dataset.Dataset)

In [ ]:
len(val_data), len(dataset)

(2, 100)

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset)
val_data = df.iloc[:20]  # Selects the first 20 rows of the DataFrame

In [ ]:
len(val_data), len(dataset)

(20, 100)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
type(val_data.iloc[0]['abstract'])

str

In [ ]:
summaries = []
references = []

In [ ]:
for i in range(len(val_data)):
    input_text = val_data.iloc[i]["article"]
    reference = val_data.iloc[i]["abstract"]

    # Tokenize and generate summary
    inputs = tokenizer(input_text, max_length=1024, truncation=True, return_tensors="pt")
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        min_length=30,
        max_length=150,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    references.append(reference)

### for larger datasets, rn size is too small for batch processing

In [ ]:
import torch

In [ ]:
batch_size = 2

for i in range(0, len(val_data), batch_size):
    batch_texts = val_data.iloc[i:i+batch_size]["article"].tolist()
    batch_refs = val_data.iloc[i:i+batch_size]["abstract"].tolist()

    # Tokenize batch
    inputs = tokenizer(batch_texts, max_length=1024, padding=True, truncation=True, return_tensors="pt")

    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        model = model.to("cuda")

    # Generate summaries for the batch
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        min_length=30,
        max_length=150,
        early_stopping=True
    )

    # Decode all summaries in batch
    batch_summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]

    summaries.extend(batch_summaries)
    references.extend(batch_refs)

KeyboardInterrupt: 

In [ ]:
summaries[0], references[0]

('we study the anchoring and phase behavior of nematic liquid crystals in contact with patterned substrates . <n> we determine an effective free energy functional for the nematic liquid crystal cell under consideration . <n> this functional takes into account the finite decay length of the nematic texture due to the surface pattern . <n> we find remarkably good agreement between the phase diagrams of various systems calculated using this effective free energy function on the one hand and the original free energy functional on the other hand .',
 'we study the phase behavior of a nematic liquid crystal confined between a flat substrate with strong anchoring and a patterned substrate whose structure and local anchoring strength we vary . by first evaluating an effective surface free energy function characterizing the patterned substrate we derive an expression for the effective free energy of the confined nematic liquid crystal . \n then we determine phase diagrams involving a homogeneou

In [ ]:
  import nltk
  nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def postprocess_text(texts):
    return ["\n".join(nltk.sent_tokenize(text.strip())) for text in texts]

In [ ]:
summaries = postprocess_text(summaries)

In [ ]:
references = postprocess_text(references)

In [ ]:
results = rouge.compute(predictions=summaries, references=references, use_stemmer=True)

In [ ]:
results = {k: round(v * 100, 4) for k, v in results.items()}

In [ ]:
print("Evaluation results:")
for key, value in results.items():
    print(f"{key}: {value}")

Evaluation results:
rouge1: 4424.59
rouge2: 1779.9
rougeL: 2708.78
rougeLsum: 4073.24


### Try evals with fine tuned model on pegasus arxiv dataset

In [ ]:
# Load base model and tokenizer
model_name = "google/pegasus-arxiv"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
summaries_peg_arxiv = []
references_peg_arxiv = []

In [ ]:
for i in range(len(val_data)):
    input_text = val_data.iloc[i]["article"]
    reference = val_data.iloc[i]["abstract"]

    # Tokenize and generate summary
    inputs = tokenizer(input_text, max_length=1024, truncation=True, return_tensors="pt")
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        min_length=30,
        max_length=150,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries_peg_arxiv.append(summary)
    references_peg_arxiv.append(reference)

In [ ]:
summaries_peg_arxiv[0], references_peg_arxiv[0]

('we study the anchoring and phase behavior of nematic liquid crystals in contact with patterned substrates . <n> we determine an effective free energy functional for the nematic liquid crystal cell under consideration . <n> this functional takes into account the finite decay length of the nematic texture due to the surface pattern . <n> we find remarkably good agreement between the phase diagrams of various systems calculated using this effective free energy function on the one hand and the original free energy functional on the other hand .',
 'we study the phase behavior of a nematic liquid crystal confined between a flat substrate with strong anchoring and a patterned substrate whose structure and local anchoring strength we vary . by first evaluating an effective surface free energy function characterizing the patterned substrate we derive an expression for the effective free energy of the confined nematic liquid crystal . \n then we determine phase diagrams involving a homogeneou

In [ ]:
summaries_peg_arxiv = postprocess_text(summaries_peg_arxiv)
references_peg_arxiv = postprocess_text(references_peg_arxiv)

In [ ]:
results = rouge.compute(predictions=summaries_peg_arxiv, references=references_peg_arxiv, use_stemmer=True)

In [ ]:
results = {k: round(v * 100, 4) for k, v in results.items()}

In [ ]:
print("Evaluation results:")
for key, value in results.items():
    print(f"{key}: {value}")

Evaluation results:
rouge1: 44.2459
rouge2: 17.799
rougeL: 27.0878
rougeLsum: 40.7324


### LED

In [ ]:
from transformers import LEDForConditionalGeneration, LEDTokenizer

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")
model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv").to("cuda").half()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

In [ ]:
def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=16384, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")

  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_length=512, num_beams=4)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch



In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from datasets import load_dataset

In [ ]:
# Load your evaluation data
dataset = load_dataset("ccdv/arxiv-summarization", split="validation[:100]")  # replace with your dataset

dataset

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00003-of-00015.parquet:   0%|          | 0.00/227M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'abstract'],
    num_rows: 100
})

In [ ]:
dataset_small = dataset.select(range(20))

In [ ]:
result_small = dataset_small.map(generate_answer, batched=True, batch_size=2)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=390667deae69743ebc08473178fb0966ca5537a0358092fbcf340adf680ac5ba
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate
import nltk

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
result_small

Dataset({
    features: ['article', 'abstract', 'predicted_abstract'],
    num_rows: 20
})

In [ ]:
results = rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"], use_stemmer=True)

In [ ]:
results = {k: round(v * 100, 4) for k, v in results.items()}
print("Evaluation results:")
for key, value in results.items():
    print(f"{key}: {value}")

Evaluation results:
rouge1: 51.9619
rouge2: 23.0691
rougeL: 30.6802
rougeLsum: 47.0503
